In [ ]:
// hacky way to set java path since macos breaks this
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
//set sys_paths to null so that java.library.path will be reevaluated next time it is needed
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
// note: make sim link between bounder and Bounder due to macos case sensitivity before hand
val path = s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
//unzip -p jdiff.jar META-INF/MANIFEST.MF
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._

import $ivy.`com.lihaoyi::scalatags:0.12.0`

import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._
import scala.collection.mutable

In [ ]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.ExpTag
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.SpecSpace
import edu.colorado.plv.bounder.lifestate.SpecSignatures
import edu.colorado.plv.bounder.symbolicexecutor.state.InitialQuery
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
import almond.interpreter.api.DisplayData

Driver.setZ3Path(s"${userhome}/software/z3/build")

var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
var android_home = android_home_possible.find(p => File(p).exists()).get             
BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))

System.setProperty("user.dir", s"${System.getProperty("user.home")}/Documents/source/bounder/notebooks/ossExp/SpecGen");
val expDir = File(System.getProperty("user.dir"))

In [ ]:
val db = new ExperimentsDb(Some(s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"))
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}

Displayers.register(classOf[Seq[TableAble]], (people: Seq[TableAble]) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(people.head.headers.map(v => th(v))),
        for (row <- people) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
          }
        )
      ).render
    }
  ).asJava
})

In [ ]:
case class JoinedResultRow(tag:String, summary:String, loc:List[String], qry:String, config:String, apk:String) extends TableAble{
    override def toString = s"JoinedResultRow(summary = ${summary}, apk = ${apk}"
    def getSummary:BounderUtil.ResultSummary = read[BounderUtil.ResultSummary](summary)
    def getConfig:RunConfig = read[RunConfig](config)
    def headers = List("tag", "summary", "qry", "apk")
    def values = List (tag  ,  summary , qry  , apk)
    def merge(other:JoinedResultRow):JoinedResultRow = {
        assert(tag == other.tag)
        assert(qry == other.qry)
        assert(config == other.config)
        assert(apk == other.apk)
        
        JoinedResultRow(tag, write[BounderUtil.ResultSummary](Driver.reduceResults(getSummary, other.getSummary)), loc ++ other.loc,qry, config, apk)
    }
}

case class SummaryRow(witnessed:Set[JoinedResultRow] = Set.empty, unreachable:Set[JoinedResultRow] = Set.empty, 
                      timeout:Set[JoinedResultRow] = Set.empty, err:Set[JoinedResultRow] = Set.empty) extends TableAble{
    def headers =  List("witnessed", "unreachable", "timeout", "err")
    def values = List(witnessed.size, unreachable.size, timeout.size, err.size)
    def add(row:JoinedResultRow):SummaryRow = {
        row.getSummary match{
            case BounderUtil.Timeout => this.copy(timeout = timeout + row)
            case BounderUtil.Witnessed => this.copy(witnessed = witnessed + row)
            case BounderUtil.Unreachable => this.copy(unreachable = unreachable + row)
            case BounderUtil.Interrupted(_) => this.copy(err = err + row)
        }
    }
}
case class ApkSummaryRow(apk:String, row:SummaryRow = SummaryRow()) extends TableAble{
    def headers = "apk"::row.headers
    def values = apk::row.values
    def add(newRow:JoinedResultRow):ApkSummaryRow = {
        assert(newRow.apk == apk)
        this.copy(row = row.add(newRow))
    }
}



implicit val getCoffeeResult = GetResult(r => JoinedResultRow(r.<<, r.<<, List(r.<<), r.<<, r.<<, r.<<))
object ReadResults{
//where cast(results.result::json->'summary' as varchar) like '%Witnessed%' or cast(results.result::json->'summary' as varchar) like '%timeout%'
    def selectResults() = {
        val q = sql"""select results.jobtag, cast(results.result::json->'summary' as varchar), results.loc, results.qry, jobs.config as cfg, cast(jobs.config::json->'apkPath' as varchar) as benchmark from results inner join jobs on jobs.id=results.jobid"""
        Await.result(db.db.run(q.as[JoinedResultRow]), 30 seconds)   
    }
    val allResults = selectResults()
    
    val rand = new scala.util.Random
    
    // Top model
    val allTopResults = allResults.filter(res => read[ExpTag](res.tag).specRefinement == "")
    val topResultsByQuery = allTopResults.groupBy(res => (res.apk,res.qry)).map{ // each query can have multiple locations, we just merge them
        case (_, rows) => rows.reduce((a:JoinedResultRow,b:JoinedResultRow) => a.merge(b))
    }.toSeq
    val topByApp = topResultsByQuery.foldLeft(Map[String,ApkSummaryRow]()){
        case (acc,joinedRow) => 
            acc + (joinedRow.apk -> acc.getOrElse(joinedRow.apk, ApkSummaryRow(joinedRow.apk)).add(joinedRow))    
    }.values.toSeq.sortBy(_.apk)
    val appList = topByApp.map{v => v.apk}
    val topByResult = Seq(topResultsByQuery.foldLeft(SummaryRow()){
        case (acc, joinedRow) => acc.add(joinedRow)
    })
    def summaryForApkTop(app:String) = {
        val res = topByApp.filter(row => row.apk.contains(app))
        assert(res.size == 1, s"Failure, wrong number of results ${res.size}")
        res.head
    }
    def sampleWitForTop(app:String) = {
        val summary = summaryForApkTop(app)
        val witnesses = summary.row.witnessed.toList
        if(witnesses.isEmpty)
            None
        else{
            val max = witnesses.size
            Some(witnesses(rand.nextInt(max)))
        }
    }
    def getTopCfgFor(app:String,clazz:String,method:String,line:Option[Int]) = {
        ???
    }
    
}
println(s"all results: ${ReadResults.allResults.size}")
println(s"all top results: ${ReadResults.allTopResults.size}")
println(s"all top results by qry: ${ReadResults.topResultsByQuery.size}")

In [ ]:
ReadResults.topByResult

In [ ]:
ReadResults.topByApp

In [ ]:
read[RunConfig](ReadResults.topByApp.head.row.witnessed.head.config).outFolder


In [ ]:
object SamplesToSpecify{
    // choose 1 witness sample per app
    def toInt(s: String): Option[Int] = {
      try {
        Some(s.toInt)
      } catch {
        case e: Exception => None
      }
    }
    def createHeuristicDir(cfg:RunConfig) = {
        val appDirName:String = cfg.outFolder.get.split("/").last
        val appDir = expDir / appDirName
        appDir.createIfNotExists(asDirectory=true)
        val heuristicDir = appDir / cfg.tag.heuristicType
        heuristicDir.createIfNotExists(asDirectory=true)
        heuristicDir
    }
    def writeSamples(count:Int) = {
        val samples = ReadResults.appList.map(app => ReadResults.sampleWitForTop(app)) //TODO: random choice needs to be moved so it gets more than one per app per run
        samples.flatten.zipWithIndex.foreach{
            case (sample,ind) => 
                val cfg = sample.getConfig
                val heuristicDir = createHeuristicDir(cfg)
                val existingDir = heuristicDir.glob("*").toList.filter(a => !a.name.startsWith("."))
                val existingConfigs = heuristicDir.glob("**/config.json").toList.map(v => read[RunConfig](v.contentAsString))
                val currentCfg = read[RunConfig](sample.config).copy(initialQuery=List(read[InitialQuery](sample.qry)))
                if(existingDir.size < count){
                    if(existingConfigs.forall(_ != currentCfg)){
                        val nextNum = if(existingDir.isEmpty) 0 else{ existingDir.flatMap(v => toInt(v.name)).max + 1 }
                        val sampleDir = heuristicDir / nextNum.toString
                        sampleDir.createIfNotExists(asDirectory=true)
                        (sampleDir / "config.json").overwrite(write[RunConfig](currentCfg))
                    }
                }
                
                // val f = expDir / s"sample_${ind}"
                // assert(!f.exists())
                // f.createIfNotExists(asDirectory=true)
                // (f / "apk").overwrite(sample.apk)
                // (f / "summary").overwrite(sample.summary)
                // (f / "configTop.json").overwrite(write[RunConfig](read[RunConfig](sample.config).copy(initialQuery=List(read[InitialQuery](sample.qry)))))
        }
    }
}
SamplesToSpecify.writeSamples(1)
println(s"Samples to specify: ${expDir.glob("**/configTop.json").toList.size}")

In [ ]:
object Sample0{
    val f = expDir / "sample_0"
}

In [ ]:
SamplesToSpecify.samples(3)

In [ ]:
SamplesToSpecify.toInt("3")

In [ ]:
List(1,2,7).max

In [ ]:
List[Int]().max